In [1]:
# !pip3 install python-Levenshtein

In [1]:
import pandas as pd
from razdel import sentenize, tokenize
from tqdm.auto import tqdm
import Levenshtein
from pymystem3 import Mystem
from string import punctuation

punct = punctuation+'«»—…“”*–'
pd.set_option('display.max_colwidth', -1)

In [2]:
from preprocess_dataset import tokenize_text, intersection, lemmatize, get_leven_span, get_leven_span_with_dist, disambiguate_span, char_span_to_word_span


# Предобработка нашего датасета

In [3]:
data = pd.read_csv("sdsj2017_sberquad.csv")

In [4]:
# эта предобработка уменьшает количество ошибок
data.answer = [i.strip().strip(".?") for i in data.answer]
data.paragraph = [i.strip() for i in data.paragraph]

In [5]:
spans = []
for (i,j) in tqdm(zip(data.answer, data.paragraph)):
    res = get_leven_span_with_dist(i, j)
    spans.append(res)
    
# соберем все возможные спаны

# Посмотрим, какие спаны получились

In [6]:
spans[:5]

[([(60, 76)], 0),
 ([(187, 193), (493, 499)], 0),
 ([(335, 358)], 0),
 ([(852, 881)], 0),
 ([(735, 755)], 0)]

In [7]:
len([i for i in spans if len(i[0]) != 0])/len(spans)*100
# у всех параграфов определился какой-то спан

100.0

In [8]:
for i, (span, dist) in enumerate(spans):
    if dist > 0:
        print(i, span, dist)
        
# у одного примера расстояние Левенштейна после lower-casing не равно 0

34157 [(0, 672), (1, 673)] 2


In [9]:
print(data.paragraph[34157], data.question[34157], data.answer[34157], sep="\n\n")

Четвёртый альбом Issues (1999) группы Korn также попал на вершину Billboard 200.[118][153] За один месяц он стал трижды платиновым.[154] За первую неделю он разошёлся тиражом 573,000 копий,[153] а первый сингл Falling Away from Me достиг 99 позиции в Billboard Hot 100.[155] Немногим раньше, до выхода альбома, Korn появляются в эпизоде Отличная загадка группы Korn о пиратском призраке мультсериала Южный парк, в котором состоялась премьера сингла Falling Away from Me .[156][157] В конце 1990-х и начале 2000-х различные ню-метал-группы, как, например, Korn,[158][159] Limp Bizkit[160][161] и P.O.D.,[162][163] стабильно участвуют в телепередаче Total Request Live на MTV.

Сколько в первую неделю было продано копий альбома Issues группы Korn

Четвёртый альбом Issues (1999) группы Korn также попал на вершину Billboard 200.[118][153] За один месяц он стал трижды платиновым.[154] За первую неделю он разошёлся тиражом 573000 копий,[153] а первый сингл Falling Away from Me достиг 99 позиции в Bil

In [10]:
# исправим ответ на правильный
data.answer[34157] = "573,000"

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
# теперь проверим еще раз
spans = []
for (i,j) in tqdm(zip(data.answer, data.paragraph)):
    res = get_leven_span_with_dist(i, j)
    spans.append(res)

In [13]:
len([i for i in spans if len(i[0]) != 0])/len(spans)*100

100.0

In [14]:
for i, (span, dist) in enumerate(spans):
    if dist > 0:
        print(i, span, dist)
# пофиксили! 

In [15]:
spans = []
for (i,j) in tqdm(zip(data.answer, data.paragraph)):
    res = get_leven_span(i, j) # теперь не будем возвращать расстояние Левенштейна
    spans.append(res)

In [16]:
len([i for i in spans if len(i) != 0])/len(spans)*100

100.0

In [17]:
spans[:20]

[[(60, 76)],
 [(187, 193), (493, 499)],
 [(335, 358)],
 [(852, 881)],
 [(735, 755)],
 [(529, 578)],
 [(553, 573)],
 [(435, 441)],
 [(526, 550)],
 [(379, 431)],
 [(115, 124)],
 [(267, 299)],
 [(10, 17), (70, 77), (113, 120), (310, 317)],
 [(415, 425)],
 [(265, 304)],
 [(151, 160)],
 [(274, 281)],
 [(565, 608)],
 [(333, 345)],
 [(281, 292)]]

### теперь нужно решить эту проблему: 

In [18]:
print(len([i for i in spans if len(i) > 1])/len(spans)*100, "% примеров имеют 2 и более спанов")

5.795806528472719 % примеров имеют 2 и более спанов


In [19]:
len([i for i in spans if len(i) > 1])

2919

In [22]:
new_spans = disambiguate_span(spans, data.paragraph, data.question, data.answer, m=Mystem())

NameError: name 'tqdm' is not defined

In [21]:
new_spans[:20]

[(60, 76),
 (187, 193),
 (335, 358),
 (852, 881),
 (735, 755),
 (529, 578),
 (553, 573),
 (435, 441),
 (526, 550),
 (379, 431),
 (115, 124),
 (267, 299),
 (70, 77),
 (415, 425),
 (265, 304),
 (151, 160),
 (274, 281),
 (565, 608),
 (333, 345),
 (281, 292)]

In [22]:
len([i for i in new_spans if len(i) == 0])

30

### У 30 примеров спан не смаппился, но все равно сохраним этот вариант в датафрейме

In [27]:
new_spans_str = [str(i) for i in new_spans]
new_spans_str[:10]

['(60, 76)',
 '(187, 193)',
 '(335, 358)',
 '(852, 881)',
 '(735, 755)',
 '(529, 578)',
 '(553, 573)',
 '(435, 441)',
 '(526, 550)',
 '(379, 431)']

In [26]:
data["char_answer_span"] = new_spans_str

In [28]:
data.head()

,paragraph_id,question_id,paragraph,question,answer,char_answer_span
0,14754,60544,"Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.",Где встречаются первые упоминания о строении человеческого тела?,в Древнем Египте,"(60, 76)"
1,13859,1604,"Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов. Один из отсеков в течение долгого времени (1993—2009 годы) занимала корректирующая оптическая система (COSTAR), установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала. Поскольку все приборы, установленные после запуска телескопа, имеют встроенные системы коррекции дефекта, во время последней экспедиции стало возможно демонтировать систему COSTAR и использовать отсек для установки ультрафиолетового спектрографа.","Как называется корректирующая оптическая система, установленная на телескоп во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала?",COSTAR,"(187, 193)"
2,8310,30350,"Критики теории Вегенера поставили во главу угла вопрос о силе, двигающей континенты, и проигнорировали всё множество фактов, безусловно подтверждавших теорию. По сути, они нашли единственный вопрос, в котором новая концепция была бессильна, и без конструктивной критики отвергли основные доказательства. После смерти Альфреда Вегенера теория дрейфа материков была отвергнута, получив статус маргинальной науки, и подавляющее большинство исследований продолжали проводиться в рамках теории геосинклиналей. Правда, и ей пришлось искать объяснения истории расселения животных на континентах. Для этого были придуманы сухопутные мосты, соединявшие континенты, но погрузившиеся в морскую пучину. Это было ещё одно рождение легенды об Атлантиде. Стоит отметить, что некоторые учёные не признали вердикт мировых авторитетов и продолжили поиск доказательств движения материков. Так дю Туа (Alexander du Toit) объяснял образование гималайских гор столкновением Индостана и Евразийской плиты.",Какая теория была отвергнута после смерти Вегенера?,теория дрейфа материков,"(335, 358)"
3,8766,53270,"При нагревании кусочки янтаря становятся очень пластичными. Это свойство используют при горячем прессовании, когда измельченный в порошок натуральный янтарь рассыпают в пресс-формы. Под воздействием температуры и давления порошок превращается в вязкую массу. После охлаждения и затвердевания в фигурных формах получаются изделия из прессованного камня. Камень, полученный таким способом, получил название амброид (от англ. amber — янтарь). Янтарный порошок может быть смешан с различными красителями, что даёт возможность получать яркие контрастные цвета (белый, чёрный, красный) и оттенки близкие к натуральным (полупрозрачный зеленоватый, вишневый). При прессовании вместе с янтарным порошком могут использоваться отшлифованные кусочки янтаря различных фракций для придания эффекта мозаичности. Изделия из прессованного янтаря не уступают по красоте изделиям из монолитных камней, однако будут стоить в разы дешевле.",Чему не уступают по красоте изделия из прессованного янтаря?,изделиям из монолитных камней,"(852, 881)"
4,14719,75698,"Нисходящие дифтонги со слабым гласным /i/ и /u/ часто встречаются в португальском. Восходящие дифтонги со слабым гласным /i/ и /u/ встречаются реже, и мноие из них могут быть квалифицированы как хиатусы (напр., fé-rias [как дифтонг] и fé-ri-as [как хиатус]: оба варианта анализа грамматически допустимы). Разница между восходящими дифтонго

In [29]:
data.to_csv("sdsj2017_sberquad_with_char_spans(full).csv", index=False)

## Теперь удалим примеры с пустыми спанами ответов

In [30]:
to_del = []
for i,j in enumerate(new_spans):
    if len(j) == 0:
        to_del.append(i)
len(to_del)

30

In [31]:
print(len(data))
data.drop(to_del, inplace=True)
print(len(data))

50364
50334


In [32]:
data.reset_index(inplace=True, drop=True)

In [33]:
data.head()

,paragraph_id,question_id,paragraph,question,answer,char_answer_span
0,14754,60544,"Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.",Где встречаются первые упоминания о строении человеческого тела?,в Древнем Египте,"(60, 76)"
1,13859,1604,"Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов. Один из отсеков в течение долгого времени (1993—2009 годы) занимала корректирующая оптическая система (COSTAR), установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала. Поскольку все приборы, установленные после запуска телескопа, имеют встроенные системы коррекции дефекта, во время последней экспедиции стало возможно демонтировать систему COSTAR и использовать отсек для установки ультрафиолетового спектрографа.","Как называется корректирующая оптическая система, установленная на телескоп во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала?",COSTAR,"(187, 193)"
2,8310,30350,"Критики теории Вегенера поставили во главу угла вопрос о силе, двигающей континенты, и проигнорировали всё множество фактов, безусловно подтверждавших теорию. По сути, они нашли единственный вопрос, в котором новая концепция была бессильна, и без конструктивной критики отвергли основные доказательства. После смерти Альфреда Вегенера теория дрейфа материков была отвергнута, получив статус маргинальной науки, и подавляющее большинство исследований продолжали проводиться в рамках теории геосинклиналей. Правда, и ей пришлось искать объяснения истории расселения животных на континентах. Для этого были придуманы сухопутные мосты, соединявшие континенты, но погрузившиеся в морскую пучину. Это было ещё одно рождение легенды об Атлантиде. Стоит отметить, что некоторые учёные не признали вердикт мировых авторитетов и продолжили поиск доказательств движения материков. Так дю Туа (Alexander du Toit) объяснял образование гималайских гор столкновением Индостана и Евразийской плиты.",Какая теория была отвергнута после смерти Вегенера?,теория дрейфа материков,"(335, 358)"
3,8766,53270,"При нагревании кусочки янтаря становятся очень пластичными. Это свойство используют при горячем прессовании, когда измельченный в порошок натуральный янтарь рассыпают в пресс-формы. Под воздействием температуры и давления порошок превращается в вязкую массу. После охлаждения и затвердевания в фигурных формах получаются изделия из прессованного камня. Камень, полученный таким способом, получил название амброид (от англ. amber — янтарь). Янтарный порошок может быть смешан с различными красителями, что даёт возможность получать яркие контрастные цвета (белый, чёрный, красный) и оттенки близкие к натуральным (полупрозрачный зеленоватый, вишневый). При прессовании вместе с янтарным порошком могут использоваться отшлифованные кусочки янтаря различных фракций для придания эффекта мозаичности. Изделия из прессованного янтаря не уступают по красоте изделиям из монолитных камней, однако будут стоить в разы дешевле.",Чему не уступают по красоте изделия из прессованного янтаря?,изделиям из монолитных камней,"(852, 881)"
4,14719,75698,"Нисходящие дифтонги со слабым гласным /i/ и /u/ часто встречаются в португальском. Восходящие дифтонги со слабым гласным /i/ и /u/ встречаются реже, и мноие из них могут быть квалифицированы как хиатусы (напр., fé-rias [как дифтонг] и fé-ri-as [как хиатус]: оба варианта анализа грамматически допустимы). Разница между восходящими дифтонго

In [34]:
spans = []
for (i,j) in tqdm(zip(data.answer, data.paragraph)):
    res = get_leven_span(i, j)
    spans.append(res)

In [35]:
char_spans = disambiguate_span(spans, data.paragraph, data.question, data.answer, m = Mystem())

In [36]:
len([i for i in char_spans if len(i) == 2]) == len(char_spans)

True

Теперь переведем char_spans в word_spans

In [37]:
word_spans, tokens = char_span_to_word_span(char_spans, data.paragraph)

In [38]:
word_spans[:10]

[[7, 10],
 [28, 29],
 [51, 54],
 [128, 132],
 [141, 144],
 [77, 82],
 [105, 108],
 [54, 55],
 [74, 76],
 [57, 64]]

In [39]:
# попробуем вывести ответы
for k,(i,j) in enumerate(zip(word_spans[:100], tokens[:100])):
    print(j[i[0]:i[1]], data.answer[k])

['в', 'Древнем', 'Египте'] в Древнем Египте
['COSTAR'] COSTAR
['теория', 'дрейфа', 'материков'] теория дрейфа материков
['изделиям', 'из', 'монолитных', 'камней'] изделиям из монолитных камней
['оральные', 'и', 'назальные'] оральные и назальные
['на', 'предсказуемость', 'швейцарского', 'правового', 'порядка'] на предсказуемость швейцарского правового порядка
['суд', 'первой', 'инстанции'] суд первой инстанции
['Лазеры'] Лазеры
['эффективность', 'разработки'] эффективность разработки
['статический', 'аспект', 'речи', '—', 'предмет', 'лингвистики', 'текста'] Статический аспект речи — предмет лингвистики текста
['до', '20', 'лет'] до 20 лет
['Виннипегская', 'всеобщая', 'забастовка'] Виннипегская всеобщая забастовка
['мутации'] Мутации
['более', '1', 'мм'] более 1 мм
['аксиоматики', 'Колмогорова', '—', 'Р', '.', 'фон', 'Мизеса'] аксиоматики Колмогорова — Р. фон Мизеса
['в', 'Бретань'] В Бретань
['До', '1917'] До 1917
['Учёные', 'и', 'студенты', 'Уппсальского', 'университета'] Учёные и студ

In [40]:
for k,(i,j) in enumerate(zip(word_spans[:100], tokens[:100])):
    if j[i[0]:i[1]] == []:
        print(k, data.answer[k])
# вроде бы все смаппилось

Добавим результаты в датафрейм

In [41]:
tokens = [" ".join(text) for text in tokens]
data["paragraph_tokens"] = tokens
data.head(2)

,paragraph_id,question_id,paragraph,question,answer,char_answer_span,paragraph_tokens
0,14754,60544,"Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.",Где встречаются первые упоминания о строении человеческого тела?,в Древнем Египте,"(60, 76)","Первые упоминания о строении человеческого тела встречаются в Древнем Египте . В XXVII веке до н . э . египетский врач Имхотеп описал некоторые органы и их функции , в частности головной мозг , деятельность сердца , распространение крови по сосудам . В древнекитайской книге Нейцзин ( XI—VII вв . до н . э . ) упоминаются сердце , печень , лёгкие и другие органы тела человека . В индийской книге Аюрведа ( Знание жизни , IX-III вв . до н . э . ) содержится большой объём анатомических данных о мышцах , нервах , типах телосложения и темперамента , головном и спинном мозге ."
1,13859,1604,"Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов. Один из отсеков в течение долгого времени (1993—2009 годы) занимала корректирующая оптическая система (COSTAR), установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала. Поскольку все приборы, установленные после запуска телескопа, имеют встроенные системы коррекции дефекта, во время последней экспедиции стало возможно демонтировать систему COSTAR и использовать отсек для установки ультрафиолетового спектрографа.","Как называется корректирующая оптическая система, установленная на телескоп во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала?",COSTAR,"(187, 193)","Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов . Один из отсеков в течение долгого времени ( 1993—2009 годы ) занимала корректирующая оптическая система ( COSTAR ) , установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала . Поскольку все приборы , установленные после запуска телескопа , имеют встроенные системы коррекции дефекта , во время последней экспедиции стало возможно демонтировать систему COSTAR и использовать отсек для установки ультрафиолетового спектрографа ."


In [42]:
word_spans_str = [",".join([str(indices[0]), str(indices[1])]) for indices in word_spans]
word_spans_str[:10]

['7,10',
 '28,29',
 '51,54',
 '128,132',
 '141,144',
 '77,82',
 '105,108',
 '54,55',
 '74,76',
 '57,64']

In [43]:
data["word_answer_span"] = word_spans_str
data.head(2)

,paragraph_id,question_id,paragraph,question,answer,char_answer_span,paragraph_tokens,word_answer_span
0,14754,60544,"Первые упоминания о строении человеческого тела встречаются в Древнем Египте. В XXVII веке до н. э. египетский врач Имхотеп описал некоторые органы и их функции, в частности головной мозг, деятельность сердца, распространение крови по сосудам. В древнекитайской книге Нейцзин (XI—VII вв. до н. э.) упоминаются сердце, печень, лёгкие и другие органы тела человека. В индийской книге Аюрведа ( Знание жизни , IX-III вв. до н. э.) содержится большой объём анатомических данных о мышцах, нервах, типах телосложения и темперамента, головном и спинном мозге.",Где встречаются первые упоминания о строении человеческого тела?,в Древнем Египте,"(60, 76)","Первые упоминания о строении человеческого тела встречаются в Древнем Египте . В XXVII веке до н . э . египетский врач Имхотеп описал некоторые органы и их функции , в частности головной мозг , деятельность сердца , распространение крови по сосудам . В древнекитайской книге Нейцзин ( XI—VII вв . до н . э . ) упоминаются сердце , печень , лёгкие и другие органы тела человека . В индийской книге Аюрведа ( Знание жизни , IX-III вв . до н . э . ) содержится большой объём анатомических данных о мышцах , нервах , типах телосложения и темперамента , головном и спинном мозге .","7,10"
1,13859,1604,"Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов. Один из отсеков в течение долгого времени (1993—2009 годы) занимала корректирующая оптическая система (COSTAR), установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала. Поскольку все приборы, установленные после запуска телескопа, имеют встроенные системы коррекции дефекта, во время последней экспедиции стало возможно демонтировать систему COSTAR и использовать отсек для установки ультрафиолетового спектрографа.","Как называется корректирующая оптическая система, установленная на телескоп во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала?",COSTAR,"(187, 193)","Телескоп имеет модульную структуру и содержит пять отсеков для оптических приборов . Один из отсеков в течение долгого времени ( 1993—2009 годы ) занимала корректирующая оптическая система ( COSTAR ) , установленная во время первой экспедиции обслуживания в 1993 году для компенсации неточности изготовления главного зеркала . Поскольку все приборы , установленные после запуска телескопа , имеют встроенные системы коррекции дефекта , во время последней экспедиции стало возможно демонтировать систему COSTAR и использовать отсек для установки ультрафиолетового спектрографа .","28,29"


In [44]:
data.to_csv("sdsj2017_sberquad_with_char_and_word_spans(minus_30_examples).csv", index=False)